## 詞頻矩陣

In [1]:
ary = ['【更新】柯P：洪智坤洩漏公文案還沒看到公文　今處理',
       '留洪智坤 柯：殘障求職不易',
       '人事處議處洪智坤　柯P：不清楚議處結果']

In [2]:
import jieba
jieba.load_userdict('userdict.txt')
corpus = []
for title in ary:
    corpus.append(' '.join(jieba.cut(title)))

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/46/b7dzk4mn6g54qzptv608w7d00000gn/T/jieba.cache
Loading model cost 0.808 seconds.
Prefix dict has been built succesfully.


In [3]:
corpus

['【 更新 】 柯P ： 洪智坤 洩漏 公文案 還 沒 看到 公文 \u3000 今 處理',
 '留 洪智坤   柯 ： 殘障 求職 不易',
 '人事處 議處 洪智坤 \u3000 柯P ： 不 清楚 議處 結果']

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer =  CountVectorizer()
X = vectorizer.fit_transform(corpus)

In [5]:
print(vectorizer.get_feature_names())

['不易', '人事處', '公文', '公文案', '更新', '柯p', '殘障', '求職', '洩漏', '洪智坤', '清楚', '看到', '結果', '處理', '議處']


In [6]:
X.toarray()

array([[0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 2]], dtype=int64)

In [7]:
from sklearn.metrics.pairwise import cosine_distances
cs = cosine_distances(X)
cs

array([[0.        , 0.8232233 , 0.76429774],
       [0.8232233 , 0.        , 0.83333333],
       [0.76429774, 0.83333333, 0.        ]])

## KMeans

In [10]:
import pandas
news = pandas.read_excel('https://raw.githubusercontent.com/ywchiu/pytextmining/master/data/20150628news.xlsx', index_col = 0)

In [11]:
news.head(3)

,category,description,pubdate,title
0,社會/生活,新北市八仙水上樂園昨晚發生粉塵爆炸，新北市衛生局統計到目前為止，由救護車送醫再加上自行送醫的...,"Sun, 28 Jun 2015 07:40:00 +0800",八仙塵爆 五相關人依公共危險重傷害法辦
1,社會/生活,新北市八仙樂園昨天晚上(6/27)舉辦活動，過程中噴灑大量玉米粉而引發粉塵爆炸，根據最新統計...,"Sun, 28 Jun 2015 07:40:00 +0800",八仙樂園意外 病患持續增加中
2,財經/要聞,希臘債務違約限期逼近，資金持續外流。路透社引述三間銀行的消息指出，希臘國內有3分之1的自動櫃...,"Sun, 28 Jun 2015 07:40:00 +0800",希臘國內三分一自動櫃員機現金短缺


In [16]:
import jieba
jieba.load_userdict('userdict.txt')
titles = []
corpus = []
for idx, rec in news.iterrows():
    #print(idx, rec)
    corpus.append(' '.join(jieba.cut(rec['description'])))
    titles.append(rec['title'])

In [19]:
#corpus[0]
titles[0]

'八仙塵爆  五相關人依公共危險重傷害法辦'

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

In [21]:
X.shape

(147, 11139)

In [22]:
from sklearn.metrics.pairwise import cosine_distances
cs = cosine_distances(X)

In [23]:
cs.shape

(147, 147)

In [27]:
from sklearn.cluster import KMeans
clf = KMeans(n_clusters=5, init='k-means++', random_state=42)
group = clf.fit_predict(cs)

In [28]:
group

array([3, 1, 2, 0, 3, 0, 1, 0, 0, 0, 0, 3, 2, 0, 4, 0, 3, 0, 0, 0, 4, 3,
       0, 0, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 4, 2, 0, 1, 2, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 1,
       1, 1, 1, 3, 2, 1, 0, 3, 1, 1, 1, 0, 3, 1, 3, 3, 1, 1, 3, 0, 0, 0,
       4, 4, 1, 3, 2, 0, 4, 3, 1, 4, 3, 4, 0, 3, 4, 4, 4, 3, 4, 0, 0, 0,
       0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 4, 3, 3, 4, 4], dtype=int32)

In [32]:
import numpy
grades = numpy.array([50,60,90,80,40])
g = numpy.array([1,1,2,2,1])
grades[g == 1]

array([50, 60, 40])

In [39]:
import numpy
np_titles = numpy.array(titles)
np_titles[group ==4].tolist()

['蔡依林淚奪金曲 錦榮傳訊恭喜',
 '陳奕迅、張惠妹稱王封后  蔡依林抱回最大獎',
 '陳奕迅、莫文蔚伴侶均不知阿娜答金曲獲獎',
 '金曲26／蔡依林擒３獎大勝\u3000淚崩再挺婚姻平權',
 '金曲26／張惠妹奪歌后卻失落\u3000要世界感受彩虹力量',
 '金曲26／蔡依林淚奪最佳專輯＋完整得獎名單',
 '僅次Jolin！徐佳瑩入圍6獎全槓被封遺珠',
 '金曲最風光！蔡依林紅毯全勝又獲3獎成大贏家',
 '張惠妹3度封后  想破江蕙紀錄',
 '金曲26／陳奕迅稱王謝台灣\u3000張惠妹封后秒噴淚',
 '蔡依林呸大贏家  金曲最佳專輯獎',
 '陳奕迅二度打敗歌神  金曲歌王好嗨',
 '金曲獎完整得獎名單！阿妹封后 陳奕迅稱王',
 '第26屆金曲獎 陳奕迅奪歌王、阿妹封歌后',
 '金曲最佳國語專輯：呸',
 '《金曲26》2015金曲獎得獎名單 線上直播懶人包']